In [9]:
from collections import Counter, defaultdict
from functools import partial
import math, random

In [13]:
#Obliczenie entropi bazując na liście prawdopodobieństw klas
def entropy(class_probabilities):
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

def partition_entropy(subsets):
    total_count = sum(len(subset) for subset in subsets)

    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

def group_by(items, key_fn):
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups
#Bazując na danych ze słownika atrybutów i etykiet, zwraca słownik: wartość atrybutu -> dane wejściowe
def partition_by(inputs, attribute):
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs,attribute):
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

def classify(tree, input):

    if tree in [True, False]:
        return tree

    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  

    if subtree_key not in subtree_dict: 
        subtree_key = None              

    subtree = subtree_dict[subtree_key] 
    return classify(subtree, input)     

def build_tree_id3(inputs, split_candidates=None):

    if split_candidates is None:
        split_candidates = inputs[0][0].keys()

    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0:                  
        return False                    

    if num_falses == 0:                 
        return True                     

    if not split_candidates:            
        return num_trues >= num_falses  

    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    subtrees[None] = num_trues > num_falses

    return (best_attribute, subtrees)

def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]


if __name__ == "__main__":

    inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]

    for key in ['level','lang','tweets','phd']:
        print(key, partition_entropy_by(inputs, key))
    print()

    senior_inputs = [(input, label)
                     for input, label in inputs if input["level"] == "Senior"]

    for key in ['lang', 'tweets', 'phd']:
        print(key, partition_entropy_by(senior_inputs, key))
    print()

    print("building the tree")
    tree = build_tree_id3(inputs)
    print(tree)

    print("Junior / Java / tweets / no phd", classify(tree,
        { "level" : "Junior",
          "lang" : "Java",
          "tweets" : "yes",
          "phd" : "no"} ))

    print("Junior / Java / tweets / phd", classify(tree,
        { "level" : "Junior",
                 "lang" : "Java",
                 "tweets" : "yes",
                 "phd" : "yes"} ))

    print("Intern", classify(tree, { "level" : "Intern" } ))
    print("Senior", classify(tree, { "level" : "Senior" } ))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617

lang 0.4
tweets 0.0
phd 0.9509775004326938

building the tree
('level', {'Senior': ('tweets', {'no': False, 'yes': True, None: False}), 'Mid': True, 'Junior': ('phd', {'no': True, 'yes': False, None: True}), None: True})
Junior / Java / tweets / no phd True
Junior / Java / tweets / phd False
Intern True
Senior False


In [3]:
inputs20 = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      False),
        ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},      False),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'no','phd':'yes'},      False),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},      False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},False),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},      True)
    ]

In [5]:
for key in ['level','lang','tweets','phd']:
        print(key, partition_entropy_by(inputs20, key))
        
print("building the tree")
tree = build_tree_id3(inputs20)
print(tree)

level 0.9480764259255209
lang 0.9120787460093084
tweets 0.8640051954592312
phd 0.8781761404668604
building the tree
('tweets', {'no': ('lang', {'Java': False, 'Python': ('level', {'Mid': ('phd', {'no': True, 'yes': False, None: False}), 'Junior': True, 'Senior': ('phd', {'no': True, None: False}), None: True}), 'R': False, None: False}), 'yes': ('lang', {'R': ('level', {'Junior': ('phd', {'no': True, 'yes': False, None: False}), 'Mid': True, 'Senior': ('phd', {'no': True, None: False}), None: True}), 'Python': True, 'Java': True, None: True}), None: True})


In [6]:
inputs50 = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      False),
        ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},      False),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'no','phd':'yes'},      False),
        ({'level':'Junior','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},      False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},False),
        ({'level':'Senior','lang':'Java','tweets':'yes','phd':'no'},      True),
    ({'level':'Junior','lang':'R','tweets':'no','phd':'yes'},False),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},True),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False),
    ({'level':'Junior','lang':'Java','tweets':'no','phd':'yes'},True),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},False),
    ({'level':'Junior','lang':'R','tweets':'no','phd':'yes'},True),
    ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},False),
    ({'level':'Junior','lang':'Java','tweets':'no','phd':'yes'},False),
    ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},True),
    ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},False),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},True),
    ({'level':'Junior','lang':'Java','tweets':'no','phd':'no'},False),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},True),
    ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},False),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'no'},True),
    ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},False),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},True),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},True),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},True),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'yes'},True),
    ({'level':'Mid','lang':'Java','tweets':'no','phd':'no'},True),
    ({'level':'Senior','lang':'Python','tweets':'no','phd':'yes'},False),
    ({'level':'Mid','lang':'R','tweets':'no','phd':'yes'},True),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},False),
    ({'level':'Senior','lang':'R','tweets':'no','phd':'no'},True),
    ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},False),
    ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},True),
    ]

In [7]:
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs50, key))

print("building the tree")
tree = build_tree_id3(inputs50)
print(tree)

level 0.986067410431086
lang 0.9232515537299046
tweets 0.9529926386381586
phd 0.9130956715238512
building the tree
('phd', {'no': ('level', {'Senior': ('tweets', {'no': ('lang', {'Java': False, 'Python': True, 'R': True, None: False}), 'yes': ('lang', {'R': True, 'Java': True, None: True}), None: True}), 'Mid': True, 'Junior': ('lang', {'Python': True, 'R': True, 'Java': ('tweets', {'yes': True, 'no': False, None: False}), None: True}), None: True}), 'yes': ('lang', {'Java': ('level', {'Senior': False, 'Junior': ('tweets', {'no': True, None: False}), 'Mid': False, None: False}), 'R': ('tweets', {'yes': ('level', {'Junior': False, 'Mid': True, None: False}), 'no': ('level', {'Mid': False, 'Junior': False, None: False}), None: False}), 'Python': ('tweets', {'yes': True, 'no': ('level', {'Junior': True, 'Mid': True, 'Senior': True, None: True}), None: True}), None: False}), None: True})
